In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import pickle
import os

## Basic Association 

In [3]:
orders = pd.read_csv('src/data/order_products__prior.csv')

In [4]:
def construct_dic_matrix(df):
    order_groups = orders.groupby('order_id')
    matrix = {}
    for order_id in order_groups.groups.keys():
        cart = order_groups.get_group(order_id)['product_id'].sort_values().to_list()
        for i, el in enumerate(cart):
            if el not in matrix:
                matrix[el] = {}
            for j in cart[(i+1):]:
                matrix[el][j] = matrix[el].get(j, 0) + 1

    delkeys = []
    for i in matrix.keys():
        for k,v in matrix[i].items():
            if v == 1:
                delkeys.append((i,k))

    for i,j in delkeys:
        del matrix[i][j]

    return matrix

In [5]:
def to_sparseMatrix(matrix):
    mat = sp.dok_matrix((49689 , 49689), dtype=np.int16)

    for ix1, ixx2 in matrix.items():
        for ix2 in ixx2:
            mat[ix1, ix2] = ixx2[ix2]

    mat = mat.transpose().tocsr()
    return mat

In [6]:
%%time
MATRIX = construct_dic_matrix(orders)

CPU times: total: 13min 35s
Wall time: 13min 38s


In [7]:
sparse_matrix = to_sparseMatrix(MATRIX)

In [8]:
with open('output/BasicAsso.pkl', 'wb') as f:
    pickle.dump(sparse_matrix, f)

In [11]:
import neptune

PROJECT="yassine-mhe/CompleteBasket"
NAME="blabla"
KEY="keyblala"
API_TOKEN="api_token"

In [18]:
model["model/specs"].upload('output/BasicAsso.pkl')

In [21]:
model["model/specs"].download(destination="downloaded_artifact")

## Apriori Algorithm